# Fusion module

&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;Этот модуль нацелен на задачи, связанные с получением большего количества временных рядов снимков. Особенно актуально это становится, если необходимо следить за изменениями, которые происходят на какой-либо территории. Например, спутниковая система [Landsat](https://landsat.usgs.gov/landsat-8) может получать не более 2-х снимков за месяц, поскольку период съемки составляет 16 дней. Другая спутниковая система [Sentinel-2](https://eros.usgs.gov/sentinel-2) имеет период до 10 дней, а потому позволяет получать до 3-х снимков за один месяц. Если объединить обе системы, то можно получать до 5 снимков в месяц на одну территорию. Это позволяет не только чаще получать данные с одной территории, но и страхует, если какие-либо из снимков оказались заняты облачностью.

&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;Однако, взять просто 2 снимка, сделанных совершенно разными спутниковыми системами, и обрабатывать совместно не получится, если не произвести никакую коррекцию. Это объясняется следующими факторами:

- **различная спутниковая аппаратура:** что означает, что значения, регистрируемые датчиками на разных системах, не будут совпадать;
- **различное время съемки:** вряд ли удастся найти два снимка, сделанных разными спутниковыми системами одномоментно;
- **различное разрешение съемки**;
- **параметры съемки (такие как высота спутника, угол визирования и т.д.) не будут одинаковыми**;
- **различные диапазоны каналов, в которых работают спутники**.

&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;Этот модуль направлен всецело на утранение именно первого пункта, поскольку он является наиболее затруднительным. При этом остальные можно только компенсировать в той или иной степени. Для этого достаточно подобрать снимки, сделанные примерно в одинаковое время, которые имели бы максимально близкие параметры съемки. Если мы берем Landsat-8 и Sentinel-2, то, оказывается, они имеют достаточно похожие диапазоны, что можно считать их или одинаковыми, или вложенными друг в друга. Различное разрешение съемки используется, чтобы максимально собрать всю информацию как со снимков с низким разрешением, так и со снимков высокого разрешения.

&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;В этом модуле доступна одна функция извне:

- [fusion](#fusion "fusion");

и пара функций внутренние:

- [cubicInterpolation](#cubicInterpolation "cubicInterpolation");
- [downscaling](#downscaling "downscaling");

## Fusion

&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;Рассмотрим вкратце последовательность действий для слияния снимков Landsat и Sentinel.

**1.** Используя ATPRK подход, улучшим качество каналов снимка Sentinel **"B11"** и **"B12"**, используя каналы **"B2"**, **"B3"**, **"B4"**, **"B8"** до 10 м;

**2.** Используя ATPRK подход, улучшим качество каналов снимка Landsat **"B2"** и **"B3"**, **"B4"**, используя канал **"B8"** до 15 м;

**3.** Улучшим разрешение каналов снимка Sentinel **"B2"** и **"B3"**, **"B4"**, используя [бикубическую интерполяцию](https://en.wikipedia.org/wiki/Bicubic_interpolation) до 5 м;

**4.** Используя ATPRK подход, улучшим качество каналов снимка Landsat **"B2"** и **"B3"**, **"B4"**, полученные на втором шаге, используя каналы снимка Sentinel **"B2"** и **"B3"**, **"B4"** до 5 м;

**5.** Уменьшим разрешение каналов, полученных на четвертом шаге, до 10 м;

**6.** Найдем коэффициенты корреляции между каналами Landsat **"B5"** и **"B6"**, **"B7"** и каналом **"B8"** ($CC (L_l, L_{PAN})$). После чего найдем коэффициенты корреляции между каналами Landsat **"B5"** и **"B6"**, **"B7"** и каналами Sentinel **"B8"** и **"B11"**, **"B12"** ($CC (L_l, S_k)$).

**7.** Если очередной $CC (L_l, L_{PAN}) > CC (L_l, S_k)$, то панхроматический канал Landsat используется для улучшения качества. В противном случае используем соответствующий канал Sentinel и повторяем шаги 2-5.

Более подробно об этом можно прочитать в статье ["Fusion of Landsat 8 OLI and Sentinel-2 MSI Data"](https://ieeexplore.ieee.org/document/7894218/).

## Functions

### fusion

&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;Use Landsat and Sentinel images to fuse them and get Landsat bands in fine Sentinel resolution.

<style type="text/css">
.tg  {border-collapse:collapse;border-spacing:0;}
.tg td{font-family:Arial, sans-serif;font-size:14px;padding:10px 5px;border-style:solid;border-width:1px;overflow:hidden;word-break:normal;border-color:black;}
.tg th{font-family:Arial, sans-serif;font-size:14px;font-weight:normal;padding:10px 5px;border-style:solid;border-width:1px;overflow:hidden;word-break:normal;border-color:black;}
.tg .tg-baqh{text-align:center;vertical-align:top}
.tg .tg-2dfk{font-weight:bold;background-color:#ecf4ff;border-color:inherit;text-align:center;vertical-align:top}
.tg .tg-amwm{font-weight:bold;text-align:center;vertical-align:top}
</style>
<table class="tg">
  <tr>
    <th class="tg-2dfk">Parameteres</th>
    <th class="tg-2dfk">Type</th>
    <th class="tg-2dfk">Description</th>
  </tr>
  <tr>
    <td class="tg-amwm">LandsatImage</td>
    <td class="tg-baqh">{image}</td>
    <td class="tg-baqh"><span style="font-style:italic">Giving Landsat image. </span><span style="font-style:italic;color:rgb(254, 0, 0)">This parameter is required!</span></td>
  </tr>
  <tr>
    <td class="tg-amwm">SentinelImage</td>
    <td class="tg-baqh">{image}</td>
    <td class="tg-baqh"><span style="font-style:italic">Giving Sentinel image. </span><span style="font-style:italic;color:rgb(254, 0, 0)">This parameter is required!</span></td>
  </tr>
  <tr>
    <td class="tg-amwm">geometry</td>
    <td class="tg-baqh">{geometry}</td>
    <td class="tg-baqh"><span style="font-style:italic">Geometry object in which bounds all calculations will be done.</span> <span style="color:rgb(50, 203, 0)">Default: </span><span style="font-weight:bold;color:rgb(50, 203, 0)">geometry = 'auto'</span> <span style="font-style:italic;text-decoration:underline">(this means that polygon will contains all Sentinel or Landsat image).</span></td>
  </tr>
  <tr>
    <td class="tg-amwm">model</td>
    <td class="tg-baqh">{String}</td>
    <td class="tg-baqh"><span style="font-style:italic">The name of using model. Availabke names are: </span><span style="font-weight:bold;font-style:italic">'power'</span><span style="font-style:italic">, </span><span style="font-weight:bold;font-style:italic">'exponent'</span><span style="font-style:italic">, </span><span style="font-weight:bold;font-style:italic">'gauss'</span><span style="font-style:italic">, </span><span style="font-weight:bold;font-style:italic">'spher'</span><span style="font-style:italic">, </span><span style="font-weight:bold;font-style:italic">'powExp'</span><span style="font-style:italic">.</span> <span style="color:rgb(50, 203, 0)">Default: </span><span style="font-weight:bold;color:rgb(50, 203, 0)">model = 'powExp'</span>.</td>
  </tr>
  <tr>
    <td class="tg-amwm">iterate</td>
    <td class="tg-baqh">{number}</td>
    <td class="tg-baqh"><span style="font-style:italic">The number of repeating iterating process for calculating coefficients. Note this parameter is required if only you use following models: </span><span style="font-weight:bold;font-style:italic">'exponent'</span><span style="font-weight:normal;font-style:italic">,</span><span style="font-style:italic"> </span><span style="font-weight:bold;font-style:italic">'gauss'</span><span style="font-style:italic">, </span><span style="font-weight:bold;font-style:italic">'powExp'</span><span style="font-style:italic">.</span> <span style="color:rgb(50, 203, 0)">Default: </span><span style="font-weight:bold;color:rgb(50, 203, 0)">iterate = 'auto'</span> <span style="font-style:italic;text-decoration:underline">(this means that it would be 50 or </span><span style="font-weight:bold;font-style:italic;text-decoration:underline">null</span><span style="font-style:italic;text-decoration:underline"> depending on the model).</span></td>
  </tr>
  <tr>
    <td class="tg-amwm">coeff</td>
    <td class="tg-baqh">{list}</td>
    <td class="tg-baqh"><span style="font-style:italic">The list of coefficients for using model. Note this parameter should contains only 2 parameters for all models except </span><span style="font-weight:bold;font-style:italic">'powExp'</span><span style="font-weight:normal;font-style:italic">: in this case use 4 parameters.</span> <span style="color:rgb(50, 203, 0)">Default: </span><span style="font-weight:bold;color:rgb(50, 203, 0)">coeff = 'auto'</span> <span style="font-style:italic;text-decoration:underline">(this means that they would be all 1 or </span><span style="font-weight:bold;font-style:italic;text-decoration:underline">null</span><span style="font-style:italic;text-decoration:underline"> depending on the model).</span></td>
  </tr>
  <tr>
    <td class="tg-amwm">PSF</td>
    <td class="tg-baqh">{list}</td>
    <td class="tg-c3ow"><span style="font-style:italic">Point spread function (PSF) which using in convolution process.</span> <span style="color:rgb(50, 203, 0)">Default: </span><span style="font-weight:bold;color:rgb(50, 203, 0)">PSF = 'auto'</span> <span style="font-style:italic;text-decoration:underline">(the box 3 by 3; all elements contain weight that equal [1 / coarse resolution]).</span></td>
  </tr>
  <tr>
    <td class="tg-amwm">windowSize</td>
    <td class="tg-baqh">{number}</td>
    <td class="tg-c3ow"><span style="font-style:italic">The size of sliding window using for finding fine residual. Note that this size shoul be choosen depending on semivariogram function. This number shoul be equal the range of graphic. If you do not know correct value, miss it.</span> <span style="color:rgb(50, 203, 0)">Default: </span><span style="font-weight:bold;color:rgb(50, 203, 0)">windowSize = 'auto'</span> <span style="font-style:italic;text-decoration:underline">(this means that size will be calculated automatically).</span></td>
  </tr>
  <tr>
    <td class="tg-amwm">initValues</td>
    <td class="tg-baqh">{list}</td>
    <td class="tg-baqh"><span style="font-style:italic">The list of init values for iterating process. Note this parameter is required if only you use following models: </span><span style="font-weight:bold;font-style:italic">'exponent'</span><span style="font-style:italic">, </span><span style="font-weight:bold;font-style:italic">'gauss</span><span style="font-style:italic">', </span><span style="font-weight:bold;font-style:italic">'powExp'</span><span style="font-style:italic">.</span> <span style="color:rgb(50, 203, 0)">Default: </span><span style="font-weight:bold;color:rgb(50, 203, 0)">initValues = 'auto'</span> <span style="font-style:italic;text-decoration:underline">(this means that they would be calculated automatically or </span><span style="font-weight:bold;font-style:italic;text-decoration:underline">null</span><span style="font-style:italic;text-decoration:underline"> depending on the model).</span></td>
  </tr>
</table>

##### Return: fuse Sentinel and Landsat images.
#### Return: image

***

### cubicInterpolation

&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;Interpolate giving image using special kind of interpolator.

<style type="text/css">
.tg  {border-collapse:collapse;border-spacing:0;}
.tg td{font-family:Arial, sans-serif;font-size:14px;padding:10px 5px;border-style:solid;border-width:1px;overflow:hidden;word-break:normal;border-color:black;}
.tg th{font-family:Arial, sans-serif;font-size:14px;font-weight:normal;padding:10px 5px;border-style:solid;border-width:1px;overflow:hidden;word-break:normal;border-color:black;}
.tg .tg-baqh{text-align:center;vertical-align:top}
.tg .tg-c3ow{border-color:inherit;text-align:center;vertical-align:top}
.tg .tg-2dfk{font-weight:bold;background-color:#ecf4ff;border-color:inherit;text-align:center;vertical-align:top}
.tg .tg-7btt{font-weight:bold;border-color:inherit;text-align:center;vertical-align:top}
.tg .tg-amwm{font-weight:bold;text-align:center;vertical-align:top}
</style>
<table class="tg">
  <tr>
    <th class="tg-2dfk">Parameteres</th>
    <th class="tg-2dfk">Type</th>
    <th class="tg-2dfk">Description</th>
  </tr>
  <tr>
    <td class="tg-7btt">image</td>
    <td class="tg-c3ow">{image}</td>
    <td class="tg-c3ow"><span style="font-style:italic">Image using for interpolate operation. </span><span style="font-style:italic;color:rgb(254, 0, 0)">This parameter is required!</span></td>
  </tr>
  <tr>
    <td class="tg-7btt">interpolator</td>
    <td class="tg-c3ow">{String}</td>
    <td class="tg-c3ow"><span style="font-style:italic">Type of interpolator. It could be </span><span style="font-weight:bold;font-style:italic">'bicubic' </span><span style="font-style:italic">or </span><span style="font-weight:bold;font-style:italic">'bilinear'</span><span style="font-style:italic">. </span><span style="font-style:italic;color:rgb(254, 0, 0)">This parameter is required!</span></td>
  </tr>
  <tr>
    <td class="tg-7btt">proj</td>
    <td class="tg-c3ow">{projection}</td>
    <td class="tg-c3ow"><span style="font-style:italic">Projection of result image.</span> <span style="font-style:italic;color:rgb(254, 0, 0)">This parameter is required!</span></td>
  </tr>
  <tr>
    <td class="tg-amwm">scale</td>
    <td class="tg-baqh">{number}</td>
    <td class="tg-baqh"><span style="font-style:italic">Scale factor of result image. </span><span style="font-style:italic;color:rgb(254, 0, 0)">This parameter is required!</span></td>
  </tr>
</table>

##### Return: interpolated image.
#### Return: image

***

### downscaling

&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;Set new resolution for using image.

<style type="text/css">
.tg  {border-collapse:collapse;border-spacing:0;}
.tg td{font-family:Arial, sans-serif;font-size:14px;padding:10px 5px;border-style:solid;border-width:1px;overflow:hidden;word-break:normal;border-color:black;}
.tg th{font-family:Arial, sans-serif;font-size:14px;font-weight:normal;padding:10px 5px;border-style:solid;border-width:1px;overflow:hidden;word-break:normal;border-color:black;}
.tg .tg-baqh{text-align:center;vertical-align:top}
.tg .tg-c3ow{border-color:inherit;text-align:center;vertical-align:top}
.tg .tg-2dfk{font-weight:bold;background-color:#ecf4ff;border-color:inherit;text-align:center;vertical-align:top}
.tg .tg-7btt{font-weight:bold;border-color:inherit;text-align:center;vertical-align:top}
.tg .tg-amwm{font-weight:bold;text-align:center;vertical-align:top}
</style>
<table class="tg">
  <tr>
    <th class="tg-2dfk">Parameteres</th>
    <th class="tg-2dfk">Type</th>
    <th class="tg-2dfk">Description</th>
  </tr>
  <tr>
    <td class="tg-7btt">image</td>
    <td class="tg-c3ow">{image}</td>
    <td class="tg-c3ow"><span style="font-style:italic">The image using in downscaling process. </span><span style="font-style:italic;color:rgb(254, 0, 0)">This parameter is required!</span></td>
  </tr>
  <tr>
    <td class="tg-7btt">proj</td>
    <td class="tg-c3ow">{projection}</td>
    <td class="tg-c3ow"><span style="font-style:italic">The projection using for downscaling process</span>. <span style="font-style:italic;color:rgb(254, 0, 0)">This parameter is required!</span></td>
  </tr>
  <tr>
    <td class="tg-amwm">sizeKernel</td>
    <td class="tg-baqh">{number}</td>
    <td class="tg-baqh"><span style="font-style:italic">The scale factor of result image resolution. </span><span style="font-style:italic;color:rgb(254, 0, 0)">This parameter is required!</span></td>
  </tr>
</table>

##### Return: downscaled image.
#### Return: image

***

## Script

var Sharpening = require('users/vasilevnikolay18/Images_fusion:Sharp');

exports.fusion = function(LandsatImage, SentinelImage, geometry, model, iterate, coeff, PSF, windowSize, initValues) {
  
  var sharpDownUp = function(Pan, LandsatChannels, SentinelChannels, scaleDown, scaleUp, interpolator) {
    
    var Step2 = Sharpening.pansharpening(Pan, LandsatChannels, 
                geometry, model, iterate, coeff, PSF, windowSize, initValues);

    var proj_B2_4 =
    ee.Image(SentinelChannels).select(ee.String(ee.Image(SentinelChannels).bandNames().get(0))).projection();
    scaleDown = ee.Number(scaleDown);
    scaleUp = ee.Number(scaleUp);

    var Step3 = cubicInterpolation(SentinelChannels, interpolator, proj_B2_4, scaleDown);
    
    var Sentinel_5m = ee.Image(Step3).bandNames();
    var Landsat_15m = ee.Image(Step2).bandNames();
    var listChannel = Sentinel_5m.zip(Landsat_15m);
    
    var Landsat_10m = listChannel.map(function (block) {
      
      var sharp_image = Sharpening.pansharpening(ee.Image(Step3).select(ee.String((ee.List(block)).get(0))), 
                        ee.Image(Step2).select(ee.String((ee.List(block)).get(1))), 
                        geometry, model, iterate, coeff, PSF, windowSize, initValues);
      var proj = ee.Image(Step2).select(ee.String((ee.List(block)).get(1))).projection();
                        
      var downscaled = downscaling(sharp_image, proj, scaleUp);
      
      return downscaled;
      
    });
    
    return Landsat_10m;

  };
  
  var fineImage_step1 = ee.Image(SentinelImage).select(['B2', 'B3', 'B4', 'B8']);
  var coarseImage_step1 = ee.Image(SentinelImage).select(['B11', 'B12']);
  
  var Step1 =  Sharpening.pansharpening(fineImage_step1, coarseImage_step1, 
               geometry, model, iterate, coeff, PSF, windowSize, initValues);
               
  SentinelImage = ee.Image(SentinelImage).addBands(Step1.rename(['B11_10m', 'B12_10m']));
  
  var scaleDown = ee.Number(1/2);
  var scaleUp = ee.Number(2);
  var interpolator = 'bicubic';
  
  var LandsatChannels = ee.Image(LandsatImage).select(['B2', 'B3', 'B4']);
  var SentinelChannels = ee.Image(SentinelImage).select(['B2', 'B3', 'B4']);
  var Pan = ee.Image(LandsatImage).select(['B8']);
  
  var fine_RGB = sharpDownUp(Pan, LandsatChannels, SentinelChannels, scaleDown, scaleUp, interpolator);
  
  LandsatImage = ee.Image(LandsatImage).addBands(ee.Image(fine_RGB).rename(['B2_10m', 'B3_10m', 'B4_10m']));
  
  var Landsatlist = ee.List(['B5', 'B6', 'B7']);
  var Sentinellist = ee.List(['B8', 'B11', 'B12']);
  var Pan_Landsat = 'B8';
  var sequence = ee.List.sequence(0, 2);
  
  var fine_B567 = sequence.map(function(i) {
    
    var Land_channel = ee.Image(LandsatImage).select(ee.String(Landsatlist.get(i)));
    var Sent_channel = ee.Image(SentinelImage).select(ee.String(SentinelImage.get(i)));
    var Pan_channel = ee.Image(LandsatImage).select(ee.String(Pan_Landsat));
    
    var pair_Pan_Land = ee.Image([Land_channel, Pan_channel]);
    var pair_Land_Sent = ee.Image([Land_channel, Sent_channel]);
    var proj = Land_channel.projection();
    
    var CC_Pan_Land = pair_Pan_Land.reduceRegion({
                        reducer: ee.Reducer.pearsonsCorrelation(),
                        bestEffort: true,
                        geometry: geometry,
                        crs: proj
                      }).get('correlation');
    
    var CC_Land_Sent = pair_Land_Sent.reduceRegion({
                         reducer: ee.Reducer.pearsonsCorrelation(),
                         bestEffort: true,
                         geometry: geometry,
                         crs: proj
                       }).get('correlation');                  
    
    return ee.Algorithms.If(ee.Number(CC_Pan_Land).gt(CC_Land_Sent),
           sharpDownUp(Pan_channel, Land_channel, Sent_channel, scaleDown, scaleUp, interpolator),
           Sharpening.pansharpening(Sent_channel, Land_channel, 
           geometry, model, iterate, coeff, PSF, windowSize, initValues));
    
  });
  
  LandsatImage = ee.Image(LandsatImage).addBands(ee.Image(fine_B567).rename(['B5_10m', 'B6_10m', 'B7_10m']));
   
  return  ee.Image(LandsatImage);
  
};


var cubicInterpolation = function(Image, interpolator, proj, scale) {
  
  return Image.resample(interpolator).reproject(ee.Projection(proj).scale(scale, scale));
  
};


var downscaling = function(image, proj, sizeKernel) {
  
  image = ee.Image(image);
  proj = ee.Projection(proj);
  
  return image.reduceResolution({
           reducer: ee.Reducer.mean(),
           bestEffort: true
         })
         .reproject(proj.scale(sizeKernel, sizeKernel));
  
};